<a href="https://colab.research.google.com/github/ngupta23/pycaret_faqs/blob/main/time_series/pycaret_log_transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    from pycaret import show_versions
except ModuleNotFoundError:
    !pip install pycaret-ts-alpha

from pycaret import show_versions
show_versions()


System:
    python: 3.7.13 (default, Mar 16 2022, 17:37:17)  [GCC 7.5.0]
executable: /usr/bin/python3
   machine: Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic

Python dependencies:
          pip: 21.1.3
   setuptools: 57.4.0
      pycaret: 3.0.0
      sklearn: 1.0.2
       sktime: 0.10.1
  statsmodels: 0.13.2
        numpy: 1.21.5
        scipy: 1.7.3
       pandas: 1.3.5
   matplotlib: 3.5.1
       plotly: 5.5.0
       joblib: 1.0.1
        numba: 0.55.1
       mlflow: Not installed
     lightgbm: 3.3.2
      xgboost: 0.90
     pmdarima: 1.8.5
        tbats: Installed but version unavailable
      prophet: Not installed
      tsfresh: Not installed


In [2]:
import numpy as np
import pandas as pd
data = pd.DataFrame(range(10,100)[::-1] + np.random.randn(90),columns=['Value'])
data.tail()

,Value
85,14.154445
86,12.447347
87,8.988288
88,10.779452
89,10.622953


In [3]:
from pycaret.time_series import TSForecastingExperiment
exp = TSForecastingExperiment()

## Setup without limiting forecasts to positive values

In [4]:
exp.setup(data=data, fh=20, seasonal_period = 1, session_id = 42)

,Description,Value
0,session_id,42
1,Target,Value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Data shape,"(90, 1)"
5,Train data shape,"(70, 1)"
6,Test data shape,"(20, 1)"
7,Fold Generator,ExpandingWindowSplitter
8,Fold Number,3
9,Enforce Prediction Interval,False


In [5]:
exp.plot_model()

In [6]:
model1 = exp.create_model("exp_smooth")
final1 = exp.finalize_model(model1)
exp.plot_model(final1)

,cutoff,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,9.0,0.8480,1.1004,0.0108,0.0108,0.8804,1.0200,0.9685
1,29.0,2.1942,2.5221,0.0383,0.0394,1.7801,1.7407,0.8080
2,49.0,0.8045,0.9642,0.0218,0.0217,0.5983,0.6163,0.9742
Mean,NaN,1.2822,1.5289,0.0237,0.0240,1.0863,1.1257,0.9169
SD,NaN,0.6451,0.7045,0.0113,0.0118,0.5039,0.4651,0.0770


## Setup limiting forecasts to positive values (using log transformation)

In [7]:
exp.setup(data=data, fh=20, seasonal_period = 1, transform_target="log", session_id = 42)

,Description,Value
0,session_id,42
1,Target,Value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Data shape,"(90, 1)"
5,Train data shape,"(70, 1)"
6,Test data shape,"(20, 1)"
7,Fold Generator,ExpandingWindowSplitter
8,Fold Number,3
9,Enforce Prediction Interval,False


In [8]:
model2 = exp.create_model("exp_smooth")
final2 = exp.finalize_model(model2)
exp.plot_model(final2)

,cutoff,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,9.0,1.7171,2.0729,0.0226,0.0223,1.7828,1.9215,0.8881
1,29.0,0.9195,1.1941,0.0152,0.0153,0.7460,0.8242,0.9570
2,49.0,2.8920,3.4510,0.0818,0.0770,2.1509,2.2059,0.6694
Mean,NaN,1.8429,2.2394,0.0399,0.0382,1.5599,1.6505,0.8382
SD,NaN,0.8101,0.9289,0.0298,0.0276,0.5948,0.5957,0.1226


**Observations**
1. Forecasts are now naturally limited to positive values.